## Installing TensorFlow Decision Forests


In [6]:
!pip install tensorflow_decision_forests

In [7]:
!pip install wurlitzer

## Importing libraries

In [8]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [9]:
# Check the version of TensorFlow Decision Forests
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v0.2.1


### Load the dataset and convert it in a tf.Dataset

In [10]:

# Load a dataset into a Pandas Dataframe.
dataset_df = pd.read_csv("/content/EEG_HMC_FeatureExtraction.csv")

# Display the first 3 examples.
dataset_df.head(100)

,Subject,Sleep Stage,Epoch,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,Spectral Edge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,MeanF_Beta_F4,Spectral Edge_Beta_F4,PeakF_Beta_F4,MeanP_Theta_F4,MedianF_Theta_F4,MeanF_Theta_F4,Spectral Edge_Theta_F4,PeakF_Theta_F4,MeanP_Delta_F4,MedianF_Delta_F4,MeanF_Delta_F4,Spectral Edge_Delta_F4,PeakF_Delta_F4,MeanP_Gamma_F4,MedianF_Gamma_F4,MeanF_Gamma_F4,Spectral Edge_Gamma_F4,PeakF_Gamma_F4,MeanP_Alpha_C4,MedianF_Alpha_C4,MeanF_Alpha_C4,Spectral Edge_Alpha_C4,PeakF_Alpha_C4,MeanP_Beta_C4,MedianF_Beta_C4,MeanF_Beta_C4,Spectral Edge_Beta_C4,PeakF_Beta_C4,MeanP_Theta_C4,MedianF_Theta_C4,MeanF_Theta_C4,Spectral Edge_Theta_C4,PeakF_Theta_C4,MeanP_Delta_C4,MedianF_Delta_C4,MeanF_Delta_C4,Spectral Edge_Delta_C4,PeakF_Delta_C4,MeanP_Gamma_C4,MedianF_Gamma_C4,MeanF_Gamma_C4,Spectral Edge_Gamma_C4,PeakF_Gamma_C4,MeanP_Alpha_O2,MedianF_Alpha_O2,MeanF_Alpha_O2,Spectral Edge_Alpha_O2,PeakF_Alpha_O2,MeanP_Beta_O2,MedianF_Beta_O2,MeanF_Beta_O2,Spectral Edge_Beta_O2,PeakF_Beta_O2,MeanP_Theta_O2,MedianF_Theta_O2,MeanF_Theta_O2,Spectral Edge_Theta_O2,PeakF_Theta_O2,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,Spectral Edge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,Spectral Edge_Gamma_O2,PeakF_Gamma_O2
0,SN002,W,0,0,9,18,17,8,0,20,30,34,16,0,4,10,8,3,0,1,5,3,1,0,36,43,50,50,0,10,19,18,10,0,20,29,33,17,0,5,12,10,4,0,1,5,3,1,0,35,42,50,50,0,11,21,20,10,0,21,30,34,21,0,4,13,11,2,0,1,4,2,1,0,35,42,50,50
1,SN002,W,1,0,10,17,17,11,0,17,26,29,20,0,4,12,10,3,0,1,4,3,1,0,35,47,50,50,0,10,18,17,10,0,17,26,28,17,0,6,13,11,4,0,1,6,3,1,0,31,41,50,50,0,11,18,18,11,0,18,27,31,19,0,6,14,11,3,0,0,4,1,0,0,34,43,50,50
2,SN002,W,2,0,10,18,17,9,0,18,25,28,19,0,5,12,9,3,0,1,4,2,1,0,34,49,50,50,0,10,18,18,9,0,18,25,27,14,0,6,13,10,3,0,1,6,3,0,0,30,41,50,50,0,11,18,18,11,0,18,25,28,21,0,6,13,11,5,0,1,5,2,1,0,35,46,50,50
3,SN002,W,3,0,10,18,17,10,0,17,26,29,16,0,6,13,10,2,0,1,6,3,1,0,33,46,50,50,0,10,18,18,10,0,19,26,28,16,0,6,13,10,6,0,1,6,3,1,0,28,39,50,50,0,10,18,18,10,0,18,27,29,16,0,6,13,11,4,0,1,7,3,1,0,32,44,50,50
4,SN002,W,4,0,10,18,17,8,0,17,26,28,18,0,5,12,9,3,0,1,5,3,0,0,34,50,50,50,0,10,18,17,10,0,17,26,28,18,0,6,13,10,6,0,1,6,3,1,0,30,40,50,50,0,10,18,18,10,0,18,26,29,20,0,6,13,11,3,0,0,6,2,0,0,34,45,50,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SN002,N3,95,0,9,13,12,11,0,12,21,21,12,0,4,11,9,5,0,1,4,2,1,0,44,50,50,50,0,10,13,13,10,0,12,19,19,13,0,4,11,9,5,0,1,4,2,1,0,24,50,50,50,0,7,14,13,6,0,13,23,24,13,0,4,10,7,3,0,1,4,2,1,0,44,50,50,50
96,SN002,N3,96,0,10,13,12,10,0,11,20,20,10,0,4,11,10,3,0,1,4,2,1,0,44,50,50,50,0,9,13,13,10,0,12,19,19,12,0,4,11,8,3,0,1,4,2,0,0,24,50,50,50,0,7,13,12,6,0,12,23,23,7,0,5,9,7,5,0,1,5,3,1,0,43,50,50,50
97,SN002,N3,97,0,9,14,13,7,0,12,22,22,11,0,4,10,8,3,0,1,4,2,1,0,50,50,50,50,0,9,13,13,6,0,12,21,21,12,0,4,10,8,3,0,1,5,3,0,0,26,50,50,50,0,7,13,12,5,0,12,23,24,12,0,5,8,7,4,0,1,5,3,0,0,42,50,50,50
98,SN002,N3,98,0,9,13,12,10,0,11,20,20,10,0,4,11,9,3,0,1,4,2,1,0,44,50,50,50,0,9,13,13,12,0,12,18,18,12,0,4,11,8,2,0,1,4,2,1,0,23,50,50,50,0,8,14,13,6,0,13,23,24,12,0,4,10,7,4,0,1,5,2,0,0,42,50,50,50


In [11]:

# Name of the label column.
label = "Sleep_Stage"

classes = dataset_df[label].unique().tolist()
print(f"Label classes: {classes}")

dataset_df[label] = dataset_df[label].map(classes.index)

Label classes: ['W', 'N1', 'N2', 'N3', 'R']


In [ ]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

In [ ]:
print(train_ds_pd)

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

### Train the model

In [ ]:
%set_cell_height 300

In [ ]:

# Specify the model.
model_1 = tfdf.keras.RandomForestModel()

# Optionally, add evaluation metrics.
model_1.compile(
    metrics=["accuracy"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs.
with sys_pipes():
  model_1.fit(x=train_ds)

## Evaluate the model

In [ ]:
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

## Prepare this model for TensorFlow Serving.

In [ ]:
model_1.save("/content/my_saved_model")

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0, max_depth=5)

## Model tructure and feature importance




In [ ]:
#%set_cell_height 300
model_1.summary()

In [ ]:
# The input features
model_1.make_inspector().features()

In [ ]:
# The feature importances
model_1.make_inspector().variable_importances()

## Model Self Evaluation


In [ ]:
model_1.make_inspector().evaluation()

## Plotting the training logs



In [ ]:
#%set_cell_height 150
model_1.make_inspector().training_logs()

In [ ]:
import matplotlib.pyplot as plt

logs = model_1.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()

In [ ]:
%load_ext tensorboard
!rm -fr "/tmp/tensorboard_logs"
model_1.make_inspector().export_to_tensorboard("/tmp/tensorboard_logs")
%tensorboard --logdir "/tmp/tensorboard_logs"